# Writing Exams

- 이메일 답장하기
- 제시문 내용 요약하기
- 자신의 의견쓰기



In [1]:
import json
from typing import List, Union

from tqdm.notebook import tqdm
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser, CommaSeparatedListOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema import HumanMessage, AIMessage, StrOutputParser
import pandas as pd

In [2]:
model = ChatOpenAI(model="gpt-3.5-turbo")

## 이메일 답장하기

### 가상의 이메일 생성하기

In [4]:
def build_text_sampling_chain(desc):
    prompt_template = PromptTemplate.from_template(template=desc)
    chain = prompt_template | model | StrOutputParser()
    return chain

In [5]:
email_gen_chain = build_text_sampling_chain(desc="영어 Writing 시험에서 이메일에 답장하기에 등장 할 법한 가상의 영어 이메일 하나 만들어줘. 이름 같은 것들도 가상으로 만들어서 채워줘. 영어로")

In [7]:
email_gen_chain.invoke({})

'Subject: Re: Inquiry about the upcoming conference\n\nDear Mr. Johnson,\n\nThank you for your email inquiring about the upcoming conference on sustainable development. I appreciate your interest in participating and I am pleased to provide you with the requested information.\n\nThe conference, titled "Sustainable Solutions for a Better Future," will be held on November 15th-17th, 2022 at the Green Valley Convention Center in San Francisco. The event aims to bring together industry leaders, policymakers, and experts to discuss and exchange ideas on various topics related to sustainable development.\n\nTo register for the conference, please visit our website www.sustainablefutureconference.com and click on the "Register Now" button. Early bird registration is available until September 30th, offering a discounted rate. After that, regular registration fees will apply.\n\nRegarding accommodation, we have partnered with several hotels in the vicinity of the convention center to offer speci

### 답장 평가하기

In [8]:
def build_eval_chain(instruction, reason_desc, score_desc):
    class Evaluation(BaseModel):
        reason: str = Field(description=reason_desc)
        score: int = Field(description=score_desc)
    
    parser = JsonOutputParser(pydantic_object=Evaluation)
    format_instructions = parser.get_format_instructions()
    
    human_prompt_template = HumanMessagePromptTemplate.from_template(
                                "# Instruction: {instruction}\n"
                                "# Context: {context}\n"
                                "# User: {input}\n"
                                "{format_instructions}",
                                partial_variables={"instruction": instruction,
                                                   "format_instructions": format_instructions})
    
    prompt = ChatPromptTemplate.from_messages(
        [
            human_prompt_template,
        ])
    eval_chain = prompt | model | parser
    return eval_chain

In [9]:
email_eval_chain = build_eval_chain(instruction="User의 응답이 Context의 이메일에 대한 적절한 응답인지 추론하고 평가하라",
                                    reason_desc="User의 응답이 Context의 이메일에 대한 적절한 응답인지에 대한 추론",
                                    score_desc="User의 응답이 Context의 이메일에 대한 적절한 응답인지에 대한 점수, 0~10점")

In [10]:
email = email_gen_chain.invoke({})

In [11]:
email

'Subject: Inquiry about Upcoming Event\n\nDear Mr. Anderson,\n\nI hope this email finds you well. My name is Emily Thompson, and I am writing to inquire about the upcoming event that was mentioned in your previous email.\n\nFirstly, I would like to express my gratitude for extending an invitation to the event. However, I would appreciate it if you could provide me with some additional details. Could you please let me know the exact date, time, and location of the event? Additionally, if there are any specific requirements or dress codes, it would be helpful to have that information as well.\n\nFurthermore, I am interested in knowing the agenda for the event. Will there be any guest speakers or presentations? It would be beneficial for me to have an understanding of the topics that will be covered during the event.\n\nLastly, I would like to confirm if there are any registration procedures or RSVP requirements. Kindly inform me if I need to complete any forms or confirm my attendance in

In [12]:
user_answer = "Dear Emily Thompson, I'm pleased to provide the details of our upcoming event, which will be held on March 15th, 6:00 PM at the Grand Hall, Downtown, and includes a formal dress code, engaging presentations by renowned speakers, and kindly requests your RSVP by March 1st"

In [13]:
eval_result = email_eval_chain.invoke({"context": email, "input": user_answer})

In [14]:
eval_result

{'reason': "The user's response is a suitable and relevant answer to the email inquiry about the upcoming event. It provides the necessary details such as the date, time, location, dress code, guest speakers, and RSVP requirements.",
 'score': 10}

## 제시문 내용 요약하기

### 무작위 글 생성

In [15]:
text_gen_chain = build_text_sampling_chain(desc="영어 Writing 시험에서 단락 요약하기에 등장 할 법한 가상의 영어 단락 하나 만들어줘. 이름 같은 것들도 가상으로 만들어서 채워줘. 영어로")

In [16]:
text = text_gen_chain.invoke({})
text

'Title: The Rise of Green Energy: A Sustainable Solution for a Brighter Future\n\nIn recent years, the global population has become more aware of the pressing need for sustainable energy sources. With the alarming rise in carbon emissions and the detrimental effects of climate change, governments and industries worldwide have been compelled to shift towards green energy alternatives. One such solution gaining momentum is solar power.\n\nSolar power harnesses the sun\'s energy to generate electricity, offering a clean and renewable energy source. Leading the charge in solar innovation is Solaris Energy, a pioneering company that has revolutionized the industry with their cutting-edge solar panels. These panels, made from state-of-the-art materials, have significantly increased the efficiency of solar energy conversion. With their sleek design and exceptional durability, Solaris panels are now a common sight on rooftops around the world.\n\nTo further promote the adoption of solar power,

### 요약 평가하기

In [17]:
summarization_eval_chain = build_eval_chain(instruction="User의 응답이 Context에 대한 적절한 요약인지 추론하고 평가하라",
                                            reason_desc="User의 응답이 Context에 대한 적절한 요약인지에 대한 추론",
                                            score_desc="User의 응답이 Context에 대한 적절한 요약인지에 대한 점수, 0~10점")

In [18]:
user_answer = "The innovative advancement and increasing adoption of solar power is emerging as a green energy solution for a sustainable future."

In [19]:
summarization_eval_chain.invoke({"context": text,
                                 "input": user_answer})

{'reason': "Yes, the user's response accurately summarizes the context.",
 'score': 9}

## 자신의 의견쓰기

### 무작위 이슈 생성


In [20]:
issue_gen_chain = build_text_sampling_chain(desc="영어 Writing 시험에서 자신의 의견쓰기에 등장 할 법한 무작위 이슈 영어 단락 하나 만들어줘. 이름 같은 것들도 가상으로 만들어서 채워줘. 영어로")

In [21]:
issue = issue_gen_chain.invoke({})
issue

'One random issue that could be used in an English writing exam for expressing one\'s opinion is the topic of "Gender-Neutral Names." In recent years, there has been a growing trend of parents choosing gender-neutral names for their children. These names, such as Jordan, Taylor, and Casey, can be given to both boys and girls. Proponents argue that gender-neutral names promote equality and break free from traditional gender stereotypes. They believe that children should have the freedom to define their own identity without being confined to societal expectations. On the other hand, some people believe that gender-neutral names can cause confusion and make it difficult for others to address individuals correctly. They argue that names carry cultural and historical significance, and altering traditional naming conventions may undermine cultural heritage. Overall, the debate surrounding gender-neutral names is an interesting one, as it touches upon the concepts of identity, individuality, 

### 의견 평가하기

In [22]:
opinion_eval_chain = build_eval_chain(instruction="User의 응답이 Context에 대한 적절한 의견 주장인지 추론하고 평가하라",
                                      reason_desc="User의 응답이 Context에 대한 적절한 의견 주장인지 대한 추론",
                                      score_desc="User의 응답이 Context에 대한 적절한 의견 주장인지에 대한 점수, 0~10점")

In [23]:
user_answer = "Gender-neutral names, while fostering individuality and challenging traditional stereotypes, may also inadvertently obscure cultural heritage and create practical challenges in social interactions."

In [24]:
opinion_eval_chain.invoke({"context": issue,
                           "input": user_answer})

{'reason': "The user's response is a well-reasoned opinion that acknowledges both the positive and negative aspects of gender-neutral names. They mention that gender-neutral names foster individuality and challenge traditional stereotypes, but also highlight the potential issues of obscuring cultural heritage and creating practical challenges in social interactions.",
 'score': 9}